**Data Mining Group 4 Jupyter Notebook**

By: Assaf Bohen, Adrian Nica, TJ Jablonski

In [1]:
import requests
import zipfile
import io
import pandas as pd
from urllib.parse import urlparse
import os

def download_and_extract_csv(zip_url):
    """
    Downloads a ZIP file from `zip_url`, extracts the first .csv file found,
    and returns it as a Pandas DataFrame.
    """
    #determines if zip is found
    response = requests.get(zip_url)
    if response.status_code == 200:
        zip_content = io.BytesIO(response.content)
        
        #extract csv files
        with zipfile.ZipFile(zip_content, 'r') as zip_ref:
            for file_name in zip_ref.namelist():
                if file_name.endswith('.csv'):
                    with zip_ref.open(file_name) as csv_file:
                        df = pd.read_csv(csv_file)

                        #print success
                        parsed_url = urlparse(zip_url)
                        print(f'Successfully extracted {file_name} from {os.path.basename(parsed_url.path)}')
                        return df

        #no csv files found
        return None
    else:
        print(f"Failed to retrieve ZIP from {zip_url}")
        return None

In [2]:
#urls to zip files
csv_url_2024 = "https://cdn.sanity.io/files/jo7n4k8s/production/262f04c41d99fea692e0125c342e446782233fe4.zip/stack-overflow-developer-survey-2024.zip"
csv_url_2023 = "https://cdn.stackoverflow.co/files/jo7n4k8s/production/49915bfd46d0902c3564fd9a06b509d08a20488c.zip/stack-overflow-developer-survey-2023.zip"
csv_url_2022 = "https://info.stackoverflowsolutions.com/rs/719-EMH-566/images/stack-overflow-developer-survey-2022.zip"

#download csv files
odf_2024 = download_and_extract_csv(csv_url_2024)
odf_2023 = download_and_extract_csv(csv_url_2023)
odf_2022 = download_and_extract_csv(csv_url_2022)

Successfully extracted survey_results_public.csv from stack-overflow-developer-survey-2024.zip
Successfully extracted survey_results_public.csv from stack-overflow-developer-survey-2023.zip
Successfully extracted survey_results_public.csv from stack-overflow-developer-survey-2022.zip


In [3]:
#converts column names to uppercase and removes spacing
odf_2024.columns = odf_2024.columns.str.upper().str.replace(" ", "")
odf_2023.columns = odf_2023.columns.str.upper().str.replace(" ", "")
odf_2022.columns = odf_2022.columns.str.upper().str.replace(" ", "")

#prints all rows
with pd.option_context('display.max_columns', None,
                       'display.max_rows', None,
                       'display.width', 6000):
    print(odf_2024.head())

   RESPONSEID                      MAINBRANCH                 AGE           EMPLOYMENT REMOTEWORK   CHECK                                   CODINGACTIVITIES                                            EDLEVEL                                          LEARNCODE                                    LEARNCODEONLINE                                            TECHDOC YEARSCODE YEARSCODEPRO                DEVTYPE ORGSIZE PURCHASEINFLUENCE BUYNEWTOOL BUILDVSBUY TECHENDORSE                                            COUNTRY CURRENCY  COMPTOTAL                             LANGUAGEHAVEWORKEDWITH                             LANGUAGEWANTTOWORKWITH                                    LANGUAGEADMIRED           DATABASEHAVEWORKEDWITH      DATABASEWANTTOWORKWITH             DATABASEADMIRED                    PLATFORMHAVEWORKEDWITH                    PLATFORMWANTTOWORKWITH                           PLATFORMADMIRED                  WEBFRAMEHAVEWORKEDWITH            WEBFRAMEWANTTOWORKWITH               WEBFRA

In [4]:
#drop all cols from odf_2024 that didn't select Apples (only survey to check for bots)
print(f'odf_2024 rows before dropping bots: {odf_2024.shape[0]}')
odf_2024 = odf_2024[odf_2024['CHECK'] == 'Apples']
print(f'odf_2024 rows after dropping bots: {odf_2024.shape[0]}\n')

#finds common column names across all dfs
common_cols = set(odf_2024.columns).intersection(odf_2023.columns).intersection(odf_2022.columns)

#creates copies of dfs
df_2024 = odf_2024[list(common_cols)].copy()
df_2023 = odf_2023[list(common_cols)].copy()
df_2022 = odf_2022[list(common_cols)].copy()

#adds a year column to all dfs
df_2024["year"] = 2024
df_2023["year"] = 2023
df_2022["year"] = 2022

#combined the common columns of the dfs
combined_df = pd.concat([df_2024, df_2023, df_2022], ignore_index=True)

#prints shape and all rows
print(f"Combined DataFrame shape: {combined_df.shape}")
with pd.option_context('display.max_columns', None,
                       'display.max_rows', None,
                       'display.width', 6000):
    print(combined_df.head())

odf_2024 rows before dropping bots: 65437
odf_2024 rows after dropping bots: 65437

Combined DataFrame shape: (227889, 63)
               SOPARTFREQ           EMPLOYMENT                      TOOLSTECHWANTTOWORKWITH TBRANCH SURVEYEASE                                            EDLEVEL             OFFICESTACKASYNCWANTTOWORKWITH  RESPONSEID TIMEANSWERING           SURVEYLENGTH PURCHASEINFLUENCE FREQUENCY_3 FREQUENCY_2 SOACCOUNT         NEWCOLLABTOOLSHAVEWORKEDWITH CURRENCY KNOWLEDGE_1 FREQUENCY_1 OPSYSPROFESSIONALUSE PROFESSIONALTECH OFFICESTACKSYNCWANTTOWORKWITH                     OFFICESTACKASYNCHAVEWORKEDWITH                                          LEARNCODE  CONVERTEDCOMPYEARLY YEARSCODEPRO         OPSYSPERSONALUSE KNOWLEDGE_2                    PLATFORMWANTTOWORKWITH                                         NEWSOSITES                           MISCTECHWANTTOWORKWITH                 KNOWLEDGE_6                DEVTYPE                    PLATFORMHAVEWORKEDWITH YEARSCODE            WEBF

**Compensation Data Exploration**

In [5]:
#exchange rates as of 2024 https://fiscaldata.treasury.gov/currency-exchange-rates-converter/
exchange_rates = {
    'EUR': 0.954,
    'USD': 1.0,
    'INR': 0.012,
    'GBP': 0.797,
    'CAD': 0.695,
    'UAH': 0.024,
    'PLN': 0.243,
    'BRL': 0.162,
    'AUD': 0.620,
    'SEK': 0.091,
    'CHF': 1.105,
    'RUB': 0.00926,
    'CZK': 0.042,
    'ILS': 0.274,
    'DKK': 0.139,
    'NOK': 0.088,
    'TRY': 0.028,
    'NZD': 0.561,
    'MXN': 0.048,
    'ZAR': 0.053,
    'HUF': 0.00253,
    'RON': 0.209,
    'IRR': 0.0000238,
    'PKR': 0.0036,
    'BGN': 0.532,
    'CNY': 0.137,
    'JPY': 0.00638,
    'BDT': 0.0084,
    'IDR': 0.0000622,
    'TWD': 0.0306,
    'VND': 0.0000392,
    'ARS': 0.00095
}

#show changes
print(f'Rows before dropping low currencies: {odf_2024.shape[0]}')

#filter out currencies with less than 150
currency_2024 = odf_2024.copy()
currency_count = currency_2024['CURRENCY'].value_counts()
valid_currencies = currency_count[currency_count >= 150].index
currency_2024 = currency_2024[currency_2024['CURRENCY'].isin(valid_currencies)]

#show changes
print(f'Rows after dropping low currencies: {currency_2024.shape[0]}')

#create new column w first 3 letters for currency code
currency_2024['CURRENCY_CODE'] = currency_2024['CURRENCY'].str[:3]

#map exchange rates
currency_2024['EXCHANGE_RATE'] = currency_2024['CURRENCY_CODE'].map(exchange_rates)

#create new compensation column
currency_2024['COMPENSATION'] = currency_2024['COMPTOTAL'] * currency_2024['EXCHANGE_RATE']
currency_2024 = currency_2024.reset_index(drop=True)

Rows before dropping low currencies: 65437
Rows after dropping low currencies: 43660


x                       

x

**Job Satisfaction Data Exploration**

x

x

In [6]:
#show changes
print(f'Number of columns in odf_2024: {odf_2024.shape[1]}')

#columns to include in jobsat_df
jobsat_include_columns = [
    'JOBSAT',
    'JOBSATPOINTS_1', #drives team strategy
    'JOBSATPOINTS_4', #contributes to open source
    'JOBSATPOINTS_5', #secures data and environments
    'JOBSATPOINTS_6', #improves code and dev environments
    'JOBSATPOINTS_7', #learns and uses new technologies
    'JOBSATPOINTS_8', #designs and builds architecture and databases
    'JOBSATPOINTS_9', #expert that others rely on for tools/languages/processes
    'JOBSATPOINTS_10', #works with new or top-tier hardware
    'JOBSATPOINTS_11', #uses a well-supported internal network for easy observability
    'KNOWLEDGE_1', #interacts with people outside immediate team
    'KNOWLEDGE_2', #knowledge silos make it hard to share ideas across org
    'KNOWLEDGE_3', #can find up-to-date info to do job effectively
    'KNOWLEDGE_4', #quickly finds answers using current tools/resources
    'KNOWLEDGE_5', #knows which system/resource to use for questions
    'KNOWLEDGE_6', #often repeats answers to the same questions
    'KNOWLEDGE_7', #delays in getting answers interrupt workflow
    'KNOWLEDGE_8', #feels equipped to work across the company's code/platform
    'KNOWLEDGE_9', #has requested employer reimbursement for helpful tools
    'FREQUENCY_1', #frequency of needing help from people outside the team
    'FREQUENCY_2', #frequency of interacting with people outside the team
    'FREQUENCY_3', #frequency of encountering knowledge silos at work
    'YEARSCODE', #coding experience including education
    'WORKEXP', #total years of work experience
    'AISELECT', #uses AI: Yes, No but plan to, No and don't plan to
    'FRUSTRATION', #challenges that cause most frustration (10 total)
    'REMOTEWORK', #work situation: Remote, in-person, remote
]

#creates jobsat_df
jobsat_df = odf_2024[jobsat_include_columns]

#show changes
print(f'Number of columns in jobsat_df: {jobsat_df.shape[1]}')
jobsat_df.head()

Number of columns in odf_2024: 114
Number of columns in jobsat_df: 27


,JOBSAT,JOBSATPOINTS_1,JOBSATPOINTS_4,JOBSATPOINTS_5,JOBSATPOINTS_6,JOBSATPOINTS_7,JOBSATPOINTS_8,JOBSATPOINTS_9,JOBSATPOINTS_10,JOBSATPOINTS_11,...,KNOWLEDGE_8,KNOWLEDGE_9,FREQUENCY_1,FREQUENCY_2,FREQUENCY_3,YEARSCODE,WORKEXP,AISELECT,FRUSTRATION,REMOTEWORK
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Remote
1,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,Agree,Agree,NaN,NaN,NaN,20,17.0,"No, and I don't plan to",NaN,Remote
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,37,NaN,"No, and I don't plan to",NaN,Remote
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4,NaN,Yes,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,9,NaN,"No, and I don't plan to",NaN,NaN


**Cleaning**

In [7]:
#drop all NaN values in JOBSAT column
print(f'Rows before dropping NaN JOBSAT values: {jobsat_df.shape[0]}')
jobsat_df = jobsat_df.dropna(subset=['JOBSAT'])
print(f'Rows after dropping NaN JOBSAT values: {jobsat_df.shape[0]}\n')

# #determines how many rows would be dropped if all NaN are dropped
# determine_drop_all = jobsat_df.copy()
# determine_drop_all = determine_drop_all.dropna()
# #shows changes
# print(f'Rows before dropping all NaN values: {jobsat_df.shape[0]}')
# print(f'Rows after dropping all NaN values: {determine_drop_all.shape[0]}')

#---only 13.1% data lost when all remaining NaNs dropped, so will just drop all remainingNaN---#
print(f'Rows before dropping all remaining NaN values: {jobsat_df.shape[0]}')
jobsat_df = jobsat_df.dropna()
print(f'Rows after dropping all remaining NaN values: {jobsat_df.shape[0]}\n')

Rows before dropping NaN JOBSAT values: 65437
Rows after dropping NaN JOBSAT values: 29126

Rows before dropping all remaining NaN values: 29126
Rows after dropping all remaining NaN values: 25320

